In [1]:
# 🔒 BULLETPROOF project + config import cell

import sys
from pathlib import Path
import os

# Detect project root by searching for config.py
current = Path.cwd()

PROJECT_ROOT = None
for parent in [current] + list(current.parents):
    if (parent / "config.py").exists():
        PROJECT_ROOT = parent
        break

if PROJECT_ROOT is None:
    raise RuntimeError("❌ config.py not found in any parent directory")

# Add project root to sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Set working directory
os.chdir(PROJECT_ROOT)

# Now import config
import config

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("📦 Config loaded from:", config.__file__)




✅ PROJECT_ROOT: c:\Users\rajku\Desktop\enterprise_knowledge_assistant
📦 Config loaded from: c:\Users\rajku\Desktop\enterprise_knowledge_assistant\config.py


In [3]:
import subprocess

from sentence_transformers import SentenceTransformer
import config
from src.vector_store.faiss_store import load_vector_store
from src.rag.citation import (
    retrieve_with_citations,
    build_rag_prompt_with_citations,
    format_citation_sources
)


In [4]:
# Load embedding model and vector store (MANDATORY SETUP)

# Load embedding model
embedding_model = SentenceTransformer(config.EMBEDDING_MODEL_NAME)
print("✅ Embedding model loaded:", config.EMBEDDING_MODEL_NAME)

# Load FAISS vector store
faiss_index, documents = load_vector_store(config.VECTOR_STORE_PATH)
print(f"✅ Loaded FAISS index with {faiss_index.ntotal} vectors")


✅ Embedding model loaded: all-MiniLM-L6-v2
✅ Vector store loaded
✅ Loaded FAISS index with 98 vectors


In [5]:
# Ollama Call

def call_llm(prompt: str) -> str:
    result = subprocess.run(
        ["ollama", "run", config.OLLAMA_MODEL_NAME],
        input=prompt.encode("utf-8"),   # 🔑 force UTF-8
        capture_output=True
    )

    return result.stdout.decode("utf-8", errors="ignore").strip()



In [6]:
# Query + Retrieval with Citations

query = "How does surface roughness affect heat transfer?"

retrieved_chunks = retrieve_with_citations(
    query=query,
    model=embedding_model,
    index=faiss_index,
    documents=documents,
    top_k=config.TOP_K
)


In [7]:
# Build RAG Prompt (Citation-Aware)

prompt = build_rag_prompt_with_citations(query, retrieved_chunks)

print("🔹 PROMPT SENT TO LLM:\n")
print(prompt)


🔹 PROMPT SENT TO LLM:

You are a knowledgeable assistant.
Answer the question using ONLY the context below.
Each factual statement MUST include citation IDs like [1], [2].
If the answer is not present in the context, say "I don't know".

Context:
[1] Thermal behavior analysis
The melting temperature of a solder interconnect is a critical parameter to ensure compatibility with the operating 
temperature of electronic devices and the thermal limits of integrated IC components. It is also imperative to 
assess the influence of sustainable reinforcements such as Agro-waste (RHA) and (WS), which are increasingly 
considered in green manufacturing. Figure 3 presents the differential scanning calorimetry (DSC) profiles of the 
plain and composite solder samples, while the associated thermal parameters, melting temperature (Tm), solidus 
temperature (Ts), liquidus temperature (TL), pasty range (ΔT), and enthalpy of fusion (ΔH) are summarized in 
Table 2.
Compared to the plain solder alloy (S0)

In [8]:
# Generate Answer (CITED)

answer = call_llm(prompt)

print("🧠 ANSWER WITH CITATIONS:\n")
print(answer)


🧠 ANSWER WITH CITATIONS:

I don't know. The provided context discusses thermal behavior analysis, including melting temperature, solidus temperature, liquidus temperature, pasty range, and enthalpy of fusion, but it does not mention surface roughness or its effect on heat transfer.


In [15]:
# Display Citation Sources

print("\n📚 SOURCES USED:\n")

for chunk in retrieved_chunks:
    print(f"{chunk['citation_id']} Distance={chunk['distance']:.4f}")
    print(chunk["content"][:400])
    print("-" * 80)



📚 SOURCES USED:

[1] Distance=1.3890
Thermal behavior analysis
The melting temperature of a solder interconnect is a critical parameter to ensure compatibility with the operating 
temperature of electronic devices and the thermal limits of integrated IC components. It is also imperative to 
assess the influence of sustainable reinforcements such as Agro-waste (RHA) and (WS), which are increasingly 
considered in green manufacturing. 
--------------------------------------------------------------------------------
[2] Distance=1.3896
moderately widened pasty ranges of 24.14 °C and 23.47 °C compared to 22.12 °C for plain solder. In particular, 
the S4 compound exhibited the broadest pasty range (27.04 °C), indicating prolonged semisolid behavior during 
cooling, which may increase the probability of porosity or hot tearing due to shrinkage and differential thermal 
contraction4.
In terms of enthalpy of fusion (ΔH), quantitative
-----------------------------------------------------------